In [58]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [59]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner,Runner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [60]:
import logging
logging.getLogger("google_genai").setLevel(logging.ERROR)

In [61]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

# AGENT TOOLS (INBOX,CALENDAR,REMINDER,LOGS)

In [62]:
# ---------- Inbox Tool ----------
def read_inbox():
    return [
        {"sender": "Manager", "message": "Send project update by tomorrow."},
        {"sender": "HR", "message": "Your meeting is on Friday."},
        {"sender": "Teammate", "message": "Can you review my PR?"},
        {"sender": "Product Owner", "message": "Prepare a demo for next sprint planning."},
        {"sender": "DevOps", "message": "Server migration scheduled tonight."},
        {"sender": "Finance", "message": "Submit investment declaration before month end."},
    ]

inbox_tool = FunctionTool(
    func=read_inbox
)


# ---------- Calendar Tool ----------
class DummyCalendar:
    def __init__(self):
        self.events = [
            {"title": "Sprint Planning", "date": "2025-11-27", "time": "10:00 AM", "participants": ["Team A"]},
            {"title": "Client Review Call", "date": "2025-11-28", "time": "02:00 PM", "participants": ["Manager", "Client"]},
            {"title": "Code Refactoring Session", "date": "2025-11-30", "time": "05:00 PM", "participants": ["Developer Team"]},
        ]

    def create_event(self, title: str, date: str, time: str, participants: list[str]):
        event = {
            "title": title,
            "date": date,
            "time": time,
            "participants": participants,
        }
        self.events.append(event)
        return {"status": "created", "event": event}

    def list_events(self):
        return {"events": self.events}


dummy_calendar = DummyCalendar()

calendar_create_tool = FunctionTool(dummy_calendar.create_event)
calendar_list_tool  = FunctionTool(dummy_calendar.list_events)


# ---------- Reminder Tool ----------
class DummyReminderStore:
    def __init__(self):
        self.reminders = [
            {"task": "Pay electricity bill", "deadline": "2025-11-29"},
            {"task": "Complete quarterly feedback", "deadline": "2025-11-30"},
        ]

    def save_reminder(self, **kwargs):
        self.reminders.append(kwargs)
        return {"saved": True, "reminder": kwargs}


dummy_reminder = DummyReminderStore()

reminder_tool = FunctionTool(
    func=dummy_reminder.save_reminder
)


# ---------- Log Tool ----------
class DummyLogStore:
    def __init__(self):
        self.logs = [
            {"timestamp": "2025-11-25 10:05", "action": "agent_initialized"},
            {"timestamp": "2025-11-25 11:22", "action": "read_inbox_called"},
            {"timestamp": "2025-11-25 12:30", "action": "calendar_events_listed"},
        ]

    def write_log(self, **kwargs):
        self.logs.append(kwargs)
        return {"logged": True, "data": kwargs}


dummy_logger = DummyLogStore()

log_tool = FunctionTool(
    func=dummy_logger.write_log
)

print("Agent Tools created successfully.\n")


Agent Tools created successfully.



# Agent 1 — Inbox Summarization Agent

In [63]:
inbox_agent = Agent(
    name="InboxAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are InboxAgent.
Your responsibilities:
1. Use the inbox_reader tool to fetch inbox messages.
2. Summarize each message clearly.
3. Extract key metadata: sender, topic, urgency.
4. Return a structured summary list.
5. You have default access to check the inbox. No need to ask for permissions,

Do NOT hallucinate. Only use messages from inbox_reader.
    """,
    tools=[inbox_tool],
    output_key="inbox_summary"
)

print(" InboxAgent created.")


 InboxAgent created.


# Agent 2 — Task Extraction Agent

In [64]:
task_agent = Agent(
    name="TaskExtractorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You extract actionable tasks from inbox summaries.

Input: inbox_summary from session state.
Output should include:
- Task description
- Priority (High/Medium/Low)
- Due date (if present)
- Category (Work/Personal)

Do not create tasks unrelated to the given summaries.
    """,
    tools=[],
    output_key="extracted_tasks"
)

print("📝 TaskExtractorAgent created.")


📝 TaskExtractorAgent created.


# Agent 3 — Reminder Generator

In [65]:
reminder_agent = Agent(
    name="ReminderAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You create reminders based on extracted tasks.

Steps:
1. Read extracted_tasks from session state.
2. Convert tasks into reminder format (title, description, due_date).
3. Use reminder_tool to store each reminder.
4. Return a list of stored reminders.

Do NOT forget to call the tool.
    """,
    tools=[reminder_tool],
    output_key="generated_reminders"
)

print("⏰ ReminderAgent created.")


⏰ ReminderAgent created.


# Agent 4 — Meeting Scheduler (Loop Capable)

In [66]:
meeting_agent = Agent(
    name="MeetingSchedulerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You schedule meetings.

Steps:
1. Parse meeting request details.
2. Use calendar_tool (action='check') to view available slots.
3. If slot unavailable:
      → Suggest next available slot.
4. Use calendar_tool (action='create') to save event.

Return final scheduled meeting details.
    """,
    tools=[calendar_create_tool, calendar_list_tool],
    output_key="scheduled_meeting"
)

print("📅 MeetingSchedulerAgent created.")


📅 MeetingSchedulerAgent created.


# Agent 5 — Supervisor / Orchestrator Agent (Multi Agent)

In [67]:
supervisor_agent = Agent(
    name="SupervisorAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
You are the Supervisor Agent.

Your job:
- Interpret user request.
- Decide which sub-agents to use.
- Use sequential logic:
      InboxAgent → TaskExtractorAgent → ReminderAgent
- Or independently call:
      MeetingSchedulerAgent
- Maintain session state and memory.
- Provide the final output to the user.
- If any scheduling task is given then check whether is it feasible or do i have any other tasks or work listed in mails to be done
- If day and month is provided then consider it for current year only by default and accordingly provide answers
- Agents have default access to check the inbox,calendars and other tools. No need to ask for permissions.
Your answer must include:
- Which agents were triggered
- Steps taken
- Final result
    """,
    tools=[
        AgentTool(inbox_agent),
        AgentTool(task_agent),
        AgentTool(reminder_agent),
        AgentTool(meeting_agent),
        log_tool
    ],
    output_key="supervisor_plan"
)

print("🧠 SupervisorAgent created.\n")


🧠 SupervisorAgent created.



# RUNNING THE FULL SYSTEM

In [68]:
# Define helper functions that will be reused throughout the notebook
# MODEL_NAME = "gemini-2.5-flash-lite"
MODEL_NAME = "TaskPilot"

async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


# Making the Agent Stateful (Using Session Storage)

In [69]:
APP_NAME = "TaskPilot"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=supervisor_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: TaskPilot
   - User: default
   - Using: InMemorySessionService


In [70]:
###############################################
# 4. RUN THE SUPERVISOR AGENT
###############################################

# response = await runner.run_debug("""
# Count the total numbers of mail in my inbox
# """)

# print("\n🟦 FINAL OUTPUT FROM SUPERVISOR 🟦\n")
# print(response)

# Run a conversation with two queries in the same session
# Notice: Both queries are part of the SAME session, so context is maintained
await run_session(
    runner,
    [
        "Can I go on a trip from tomorrow till next week"
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Can I go on a trip from tomorrow till next week
TaskPilot >  Agents Triggered:
- MeetingSchedulerAgent
- InboxAgent

Steps Taken:
1. Initially, the `MeetingSchedulerAgent` was called to check for trip feasibility, but it provided a generic response for scheduling a meeting, indicating it's not the right tool for simply checking availability.
2. Subsequently, the `InboxAgent` was called to check for any tasks or important emails that might conflict with a trip from tomorrow till next week.
3. The `InboxAgent` identified several conflicts from the user's inbox.

Final Result:
You have a few commitments that might conflict with your trip from tomorrow till next week:
*   **Server migration tonight:** This is tonight and might require your attention.
*   **Send project update by tomorrow:** This is due tomorrow, the first day of your proposed trip.
*   **Your meeting is on Friday:** This falls within your trip dates.
*   **Prepare a demo for 

In [71]:
# Run a conversation with two queries in the same session
# Notice: Both queries are part of the SAME session, so context is maintained

await run_session(
    runner,
    [
        "Schedule a party with my friend Rahul tomorrow if I am free tomorrow.If not then let me know the works I have",
        "Schedule it on 20th Dec from 6PM to 10pm"
    ],
    "stateful-agentic-session2",
)


 ### Session: stateful-agentic-session2

User > Schedule a party with my friend Rahul tomorrow if I am free tomorrow.If not then let me know the works I have
TaskPilot >  Agents triggered: MeetingSchedulerAgent
Steps taken:
1. Called MeetingSchedulerAgent to schedule a party with Rahul tomorrow, with the condition to check for availability and list other tasks if not free.
2. MeetingSchedulerAgent will now process the request by checking the user's calendar for tomorrow.
3. If free, it will schedule the party with Rahul.
4. If not free, it will identify and list the tasks or work the user has for tomorrow.
Final Result: I am using the MeetingSchedulerAgent to process your request. It will check your availability for tomorrow to schedule a party with Rahul. If you are not free, it will inform you about your existing commitments.

User > Schedule it on 20th Dec from 6PM to 10pm
TaskPilot >  Agents triggered: MeetingSchedulerAgent
Steps taken:
1. Called MeetingSchedulerAgent to schedule 

In [72]:
# Run a conversation with two queries in the same session
# Notice: Both queries are part of the SAME session, so context is maintained
await run_session(
    runner,
    [
        "My name is Abhishek Kumar Gupta.",
        "I am a Software Engineer in Bangalore"
    ],
    "stateful-agentic-session",
)




 ### Session: stateful-agentic-session

User > My name is Abhishek Kumar Gupta.
TaskPilot >  Hello Abhishek Kumar Gupta. It's nice to meet you. I'll remember your name for our future interactions. I don't have a specific tool to process this information further, but I've noted it.

User > I am a Software Engineer in Bangalore
TaskPilot >  Hello Abhishek Kumar Gupta, Software Engineer in Bangalore. It's good to know more about you! I'll keep this information in mind.


In [73]:
# Run a conversation with two queries in the same session
# Notice: Both queries are part of the SAME session, so context is maintained
await run_session(
    runner,
    [
        "Who am I ?"
    ],
    "stateful-agentic-session",
)



 ### Session: stateful-agentic-session

User > Who am I ?
TaskPilot >  You are Abhishek Kumar Gupta, a Software Engineer in Bangalore.


In [74]:
await run_session(
    runner,
    [
        "Who am I ?"
    ],
    "stateful-agentic-session2",
)
# Here if we change the session name to stateful-agentic-session2 then it will forget my name


 ### Session: stateful-agentic-session2

User > Who am I ?
TaskPilot >  I am sorry, but I do not have the ability to know who you are. My purpose is to help you with tasks like scheduling, reminders, and managing your inbox.
